<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jyp/skip_gram_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
class skip_gram:
    def __init__(self,token_ls):
        self.token_ls=token_ls
        self.word_ls=sorted(list(set(self.token_ls)))

    def connected_token(self,token,window_size):
        token_index_ls=[i for i,v in enumerate(self.token_ls) if v==token]
        return_set={self.word_ls.index(v) for i,v in enumerate(self.token_ls) for j in token_index_ls if abs(i-j)<=window_size}-{self.word_ls.index(token)}
        return sorted(list(return_set))

    

In [3]:
L=["딸기","바나나","사과","딸기","파인애플","포도","블루베리"]
print(sorted(skip_gram(L).word_ls))
skip_gram(L).connected_token("딸기",2)

['딸기', '바나나', '블루베리', '사과', '파인애플', '포도']


[1, 3, 4, 5]

In [4]:
n=len(skip_gram(L).word_ls)
X=[[np.identity(n)[i] for j in skip_gram(L).connected_token(skip_gram(L).word_ls[i],2)] for i in range(n)]
X=sum(X,[])
y=[[np.identity(n)[j] for j in skip_gram(L).connected_token(skip_gram(L).word_ls[i],2)] for i in range(n)]
y=sum(y,[])
X,y=np.array(X),np.array(y)
X_1=np.column_stack((np.ones(len(X)),X))

In [14]:
learning_rate=0.003
num_epoch=400000
np.random.seed(0)
hidden_num=5

W_1=np.random.random(size=(n+1,hidden_num))
W_2=np.random.random(size=(hidden_num+1,n))

for epoch in range(num_epoch):
    first_layer=np.dot(X_1,W_1)
    output_layer=np.dot(np.column_stack((np.ones(len(first_layer)),first_layer)),W_2)
    hyp=(np.exp(output_layer).T/np.exp(output_layer).sum(axis=1)).T
    error=-np.multiply(y,np.log(hyp)).sum()/len(X)
    
    len_neighbor=[len(skip_gram(L).connected_token(skip_gram(L).word_ls[j],2)) for j in range(n)]
    arr=np.zeros(n)
    for p,v in enumerate(np.argsort(-hyp)[[sum(len_neighbor[:j]) for j in range(n)]]):
        arr=np.row_stack((arr,np.identity(n)[v<len_neighbor[p]]))
    pred=arr[1:]
    accuracy=(y.argmax(axis=1)==pred.argmax(axis=1)).mean()
    if epoch%10000==0:
        print(f"epoch : {epoch} loss : {error}")
    if error<1.13:
        break
    W_1=W_1-np.dot(X_1.T,np.dot(hyp-y,W_2.T).T[1:].T)*learning_rate/len(X)
    W_2=W_2-np.dot(np.column_stack((np.ones(len(first_layer)),first_layer)).T,(hyp-y))*learning_rate/len(X)
print(f"epoch : {epoch}, loss : {error}")

epoch : 0 loss : 2.3359386450824546
epoch : 10000 loss : 1.298963308823089
epoch : 20000 loss : 1.1678928877219374
epoch : 30000 loss : 1.1507192452689687
epoch : 40000 loss : 1.1451343041019986
epoch : 50000 loss : 1.1425145837297292
epoch : 60000 loss : 1.1410315280272585
epoch : 70000 loss : 1.1400905552294127
epoch : 80000 loss : 1.1394461499634276
epoch : 90000 loss : 1.1389800566425556
epoch : 100000 loss : 1.1386288243908371
epoch : 110000 loss : 1.1383555726676238
epoch : 120000 loss : 1.1381374997832523
epoch : 130000 loss : 1.1379597988419126
epoch : 140000 loss : 1.137812462087333
epoch : 150000 loss : 1.1376884968777072
epoch : 160000 loss : 1.1375828783433386
epoch : 170000 loss : 1.137491907995456
epoch : 180000 loss : 1.1374128065430786
epoch : 190000 loss : 1.1373434472439574
epoch : 200000 loss : 1.1372821764964107
epoch : 210000 loss : 1.1372276902218788
epoch : 220000 loss : 1.1371789468729723
epoch : 230000 loss : 1.1371351050519047
epoch : 240000 loss : 1.137095478

In [15]:
(y.argmax(axis=1)==pred.argmax(axis=1)).mean()

0.6666666666666666

In [ ]:
hyp

array([[5.79751023e-05, 2.49982579e-01, 2.69608210e-06, 2.49954897e-01,
        2.49980913e-01, 2.50020940e-01],
       [5.79751023e-05, 2.49982579e-01, 2.69608210e-06, 2.49954897e-01,
        2.49980913e-01, 2.50020940e-01],
       [5.79751023e-05, 2.49982579e-01, 2.69608210e-06, 2.49954897e-01,
        2.49980913e-01, 2.50020940e-01],
       [5.79751023e-05, 2.49982579e-01, 2.69608210e-06, 2.49954897e-01,
        2.49980913e-01, 2.50020940e-01],
       [4.99939245e-01, 6.65161945e-05, 5.14809007e-05, 4.99901163e-01,
        6.18045423e-07, 4.09767322e-05],
       [4.99939245e-01, 6.65161945e-05, 5.14809007e-05, 4.99901163e-01,
        6.18045423e-07, 4.09767322e-05],
       [6.11379375e-07, 4.63039099e-05, 6.52942722e-05, 4.23956089e-05,
        4.99946121e-01, 4.99899274e-01],
       [6.11379375e-07, 4.63039099e-05, 6.52942722e-05, 4.23956089e-05,
        4.99946121e-01, 4.99899274e-01],
       [3.33281329e-01, 3.33267667e-01, 3.57556519e-05, 6.47736640e-05,
        3.33349495e-01, 

In [ ]:
import tensorflow 
from tensorflow.keras import layers,models,Sequential
model=Sequential([
                 layers.Dense(3,input_shape=(n,),activation='linear'),
                 layers.Dense(n,activation='softmax')
                 ])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X,y,epochs=20)